Use a YOLOv5 model to detect snakes in images and crop the image based on the bounding box. Than replace image with the new images. In the process save the width and height of the image and also xmin, ymin, xmax and ymax

# Setup

In [ ]:
import torch
import tarfile
from PIL import Image,ImageFile, ImageDraw,ImageFont
#https://stackoverflow.com/questions/60584155/oserror-image-file-is-truncated
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torchvision.transforms as transforms 
from psutil import virtual_memory
import shutil
import os
import gc
from matplotlib import pyplot as plt
import numpy as np
import cv2
from matplotlib.pyplot import figure
import pandas as pd
from tqdm import tqdm
%matplotlib inline
from IPython.display import display
#from IPython.display import Image

from timm.models import create_model
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import Dataset
import math
from torchvision.transforms import ToTensor

# Raid

In [ ]:
%cd /raid/USER

In [ ]:
# Remove old files once completely, so that the image files are clean reloaded.
!rm -r images/all/SnakeCLEF2022-test_images
!rm -r images/all/SnakeCLEF2022-large_size

In [ ]:
# Reload data
TRAIN_DATA_DIR = 'OUR_PATH/images/all/SnakeCLEF2022-large_size.tar.gz'
shutil.copy(TRAIN_DATA_DIR,'/raid/USER/images/all/') 

TEST_DATA_DIR = 'OUR_PATH/images/all/SnakeCLEF2022-test_images'
shutil.copytree(TEST_DATA_DIR,'/raid/USER/images/all/SnakeCLEF2022-test_images') 

In [ ]:
# UNPACK
file = tarfile.open('./images/all/SnakeCLEF2022-large_size.tar.gz')
  
# extracting file
file.extractall('./images/all/')
  
file.close()

In [ ]:
!rm images/all/SnakeCLEF2022-large_size.tar.gz

In [ ]:
path_to_model = './saved_runs/OUR_MODEL/weights/best.pt'
path_to_train_images = './images/all/SnakeCLEF2022-large_size/'
path_to_test_images = './images/all/SnakeCLEF2022-test_images/SnakeCLEF2022-large_size/'
path_to_meta_train = './images/all/SnakeCLEF2022-TrainMetadata.csv'
path_to_meta_test = './images/all/SnakeCLEF2022-TestMetadata.csv'
path_to_new_meta = './images/all/'
path_to_archive = './images/all/'

## Pipeline
Take a picture, open it, find the snake, cut the picture based on the bounding box you found. Return the image, the snake images, the confidence and the bounding boxeen.

In [ ]:
class OD_Pipeline:
    
    def __init__(self, model_path,force_reload=True, image_size = 380):
        self.model_path = model_path
        self.image_size = image_size
        try:
            self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path,force_reload=force_reload)
            device = ("cuda" if torch.cuda.is_available() else "cpu")
            self.model.to(device)
        except Exception as e:
            print(e)
    
    def pipeline(self, image_path, resize):

        results = self.model(image_path)
        results = results.pandas().xyxy[0]
        # print(results)

        img = Image.open(image_path)
        snake_images = []
        confidence = []
        bboxs = []
        for index, bbox in results.iterrows():
            confidence.append(bbox[4])
            s_img = img.crop((int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])))
            if resize:
                s_img = s_img.resize((self.image_size,self.image_size))
            snake_images.append(s_img)
            bboxs.append([(int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3]))])

        return (img, snake_images, confidence, bboxs)

## Use OD Pipeline

In [ ]:
# train meta
df_train = pd.read_csv(path_to_meta_train)
df_train.head()

In [ ]:
# change file_path
df_train.file_path = path_to_train_images + df_train.file_path
df_train.head()

In [ ]:
# test meta
df_test = pd.read_csv(path_to_meta_test)
df_test.head()

In [ ]:
df_test.file_path = path_to_test_images + df_test.file_path

In [ ]:
# check GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# Create Pipeline object
snake_detecting_pipeline = OD_Pipeline(path_to_model,force_reload=True)

In [ ]:
df_no_snakes = pd.DataFrame()
df_error_images = pd.DataFrame()

In [ ]:
# Store all important information
df_train_prepaired = pd.DataFrame(columns=['observation_id', 'endemic', 'binomial_name', 'country', 'code',
       'class_id', 'file_path','img_width','img_height','xmin','ymin','xmax','ymax','confidence'])

In [ ]:
# There is an error with one image there the ending is missing. We added manually.
!mv ./images/all/SnakeCLEF2022-large_size/2014/Hypsiglena_ochrorhynchus/813384. ./images/all/SnakeCLEF2022-large_size/2014/Hypsiglena_ochrorhynchus/813384.jpg

In [ ]:
import time

In [ ]:
start = time.time()
for index, row in tqdm(df_train.iterrows()):
    #print(row)
    image_path = row['file_path']

    error = False
    
    try:
        # Resize was omitted to provide more flexibility for later image manipulation.
        img, snake_images, arr_confidence, bboxs = snake_detecting_pipeline.pipeline(image_path,resize=False)
    except Exception as e:
        df_error_images = df_error_images.append([image_path])
        error = True
        #print('#',e)
        
    if not error:
        # Find max conf and save only this bbox
        max_value_pos = -1
        if len(arr_confidence) > 0:
            max_value_in_arr = max(arr_confidence)

            for i in range(0,len(arr_confidence)):
                if arr_confidence[i] == max_value_in_arr:
                    max_value_pos = i
                    
                    # save image -> replace old one
                    
                    try:
                        snake_images[i].save(image_path)
                    except Exception as e:
                        print(e, image_path)
                        df_error_images = df_error_images.append([image_path])
                        error = True
                    if not error:
                        img_width, img_height = img.size
                        # save all to new meta
                        df_train_prepaired = df_train_prepaired.append({'observation_id':row['observation_id'],
                                                                        'endemic':row['endemic'],
                                                                        'binomial_name':row['binomial_name'],
                                                                        'country':row['country'],
                                                                        'code':row['code'],
                                                                        'class_id':row['class_id'], 
                                                                        'file_path':row['file_path'],
                                                                        'img_width':img_width,
                                                                        'img_height':img_height,
                                                                        'xmin':bboxs[i][0][0],
                                                                        'ymin':bboxs[i][0][1],
                                                                        'xmax':bboxs[i][0][2],
                                                                        'ymax':bboxs[i][0][3],
                                                                        'confidence':arr_confidence[i]},ignore_index=True)
                    else:
                        #error
                        df_error_images = df_error_images.append([image_path])
                    # break the loop
                    break
        else:
            df_no_snakes = df_no_snakes.append([image_path])

print('Duration: ', time.time()-start)

In [ ]:
df_train_prepaired.to_csv(path_to_new_meta+'SnakeCLEF2022_TrainMetadata_preprocessed_with_snake_detection_no_resize_OUR_NAME.csv')


In [ ]:
# new df_test
df_test_prepaired = pd.DataFrame(columns=['observation_id','endemic','country','code','file_path','img_width','img_height','xmin','ymin','xmax','ymax','confidence'])

In [ ]:
for index, row in tqdm(df_test.iterrows()):
    #print(row)
    image_path = row['file_path']

    error = False
    
    try:
        img, snake_images, arr_confidence, bboxs = snake_detecting_pipeline.pipeline(image_path,resize=False)
    except Exception as e:
        df_error_images = df_error_images.append([image_path])
        error = True
        print('#',e)
        
    if not error:
        # Find max conf and save only this bbox
        max_value_pos = -1
        if len(arr_confidence) > 0:
            max_value_in_arr = max(arr_confidence)

            for i in range(0,len(arr_confidence)):
                if arr_confidence[i] == max_value_in_arr:
                    max_value_pos = i
                    # save image -> replace old one
                    # print('scr/dst',image_path)
                    snake_images[i].save(image_path)
                    img_width, img_height = img.size
                    # save all to new meta
                    df_test_prepaired = df_test_prepaired.append({'observation_id':row['observation_id'],
                                                                    'endemic':row['endemic'],
                                                                    'country':row['country'],
                                                                    'code':row['code'],
                                                                    'file_path':row['file_path'],
                                                                    'img_width':img_width,
                                                                    'img_height':img_height,
                                                                    'xmin':bboxs[i][0][0],
                                                                    'ymin':bboxs[i][0][1],
                                                                    'xmax':bboxs[i][0][2],
                                                                    'ymax':bboxs[i][0][3],
                                                                    'confidence':arr_confidence[i]},ignore_index=True)
                    # break the loop
                    break
        else:
            df_no_snakes = df_no_snakes.append([image_path])

In [ ]:
df_test_prepaired.to_csv(path_to_new_meta+'SnakeCLEF2022_TestMetadata_preprocessed_with_snake_detection_no_resize_OUR_NAME.csv')

In [ ]:
# 'No snakes' is not a reliable table, just an indication in which images no snake was detected.
df_no_snakes.to_csv(path_to_new_meta+'No_snakes_found_in_image_OUR_NAME.csv')

In [ ]:
# Saving the images where there ma have been problemens
df_error_images.to_csv(path_to_new_meta+'Error_images_in_preprocessing_snake_OD_OUR_NAME.csv')

At this point, all images were loaded once from the selected YOLOv5 model, if possible a snake was detected and replaced as a cropped partial image in the same location. Now a classification model can be trained and used for predictions.